# Train unimodal CT

In [1]:
!git clone https://github.com/PietroCaforio/research-biocv-proj
!cd research-biocv-proj && git switch dev

Cloning into 'research-biocv-proj'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 175 (delta 91), reused 106 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (175/175), 3.40 MiB | 30.04 MiB/s, done.
Resolving deltas: 100% (91/91), done.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [2]:
!cd research-biocv-proj && git pull

Already up to date.


In [3]:
!pip install wandb

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")

In [5]:
import wandb
wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
import sys
from pathlib import Path

# Add the 'data' directory to sys.path
sys.path.append(str(Path('research-biocv-proj').resolve()))
from data.unimodal import *
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader

### Train ResNet model

In [7]:
def train(model,config, run_name=None):
  wandb.init(
    # set the wandb project where this run will be logged
    project="unimodal_ct_training",
    name = run_name,
    # track hyperparameters and run metadata
    config=config
  )
  optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])
  criterion = nn.CrossEntropyLoss()
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = config["reduce_lr_factor"], patience = config["patience"])
  # Training loop
  num_epochs = config["epochs"]
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0

      for batch in train_loader:
          frames = batch['frame'].float().to(device)
          labels = batch['label'].long().to(device)

          optimizer.zero_grad()
          outputs = model(frames)
          loss = criterion(outputs.logits, labels)

          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

      # Validation loop
      model.eval()
      val_loss = 0.0
      correct = 0
      total = 0
      # Initialize counters for each class (G1, G2, G3)
      correct_per_class = [0, 0, 0]  # For G1, G2, G3
      total_per_class = [0, 0, 0]  # For G1, G2, G3

      with torch.no_grad():
          for batch in val_loader:
              frames = batch['frame'].float().to(device)
              labels = batch['label'].long().to(device)

              outputs = model(frames)
              loss = criterion(outputs.logits, labels)

              val_loss += loss.item()
              _, predicted = torch.max(outputs.logits, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

              # Calculate accuracy per class
              for i in range(3):  # We have 3 classes: G1 (0), G2 (1), G3 (2)
                  correct_per_class[i] += ((predicted == i) & (labels == i)).sum().item()
                  total_per_class[i] += (labels == i).sum().item()
      scheduler.step(val_loss)
      # Compute total accuracy and per-class accuracy
      total_accuracy = 100 * correct / total
      class_accuracy = [(100 * correct_per_class[i] / total_per_class[i]) if total_per_class[i] > 0 else 0 for i in range(3)]
      print(f"Validation Loss: {val_loss/len(val_loader)}, Total Accuracy: {total_accuracy:.2f}%")
      print(f"Accuracy per class - G1: {class_accuracy[0]:.2f}%, G2: {class_accuracy[1]:.2f}%, G3: {class_accuracy[2]:.2f}%")
      # log metrics to wandb
      wandb.log({"Total Accuracy": total_accuracy, "Validation Loss": val_loss/len(val_loader), "G1_Acc":class_accuracy[0], "G2_Acc":class_accuracy[1], "G3_Acc":class_accuracy[2]})
  wandb.finish()  

In [8]:
import torch.nn as nn
import torch.optim as optim
from transformers import ResNetForImageClassification

In [9]:
train_dataset = UnimodalCTDataset(split='train',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/" )
val_dataset = UnimodalCTDataset(split='val',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [10]:
print(f"Training set stats:{train_dataset.stats()}")
print(f"Validation set stats:{val_dataset.stats()}")

Training set stats:{'length': 2292, 'class_frequency': {'G1': 78, 'G2': 1543, 'G3': 671}}
Validation set stats:{'length': 335, 'class_frequency': {'G1': 37, 'G2': 166, 'G3': 132}}


### Resnet-50

In [11]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-50')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [13]:
config={
    "learning_rate": 1e-4,
    "architecture": "microsoft/resnet-50",
    "epochs": 70,
    "weight_decay": 0.001,
    "reduce_lr_factor": 0.2,
    "patience": 20
    }
train(model, config, run_name = config["architecture"])

wandb: Currently logged in as: pietro-caforio (pietro-caforio-politecnico-di-milano). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240916_150705-5na9vyqd
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-50
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/5na9vyqd


Epoch 1, Loss: 0.8369050597151121
Validation Loss: 35.55460020628843, Total Accuracy: 49.55%
Accuracy per class - G1: 0.00%, G2: 100.00%, G3: 0.00%
Epoch 2, Loss: 0.5304841138422489
Validation Loss: 60.8487608730793, Total Accuracy: 49.55%
Accuracy per class - G1: 0.00%, G2: 100.00%, G3: 0.00%
Epoch 3, Loss: 0.3303564530279901
Validation Loss: 84.2468471059745, Total Accuracy: 49.25%
Accuracy per class - G1: 0.00%, G2: 88.55%, G3: 13.64%
Epoch 4, Loss: 0.18267211224883795
Validation Loss: 116.65761993486772, Total Accuracy: 66.57%
Accuracy per class - G1: 0.00%, G2: 85.54%, G3: 61.36%
Epoch 5, Loss: 0.10864931117329332
Validation Loss: 137.25224161435935, Total Accuracy: 65.37%
Accuracy per class - G1: 0.00%, G2: 83.73%, G3: 60.61%
Epoch 6, Loss: 0.07795615745190945
Validation Loss: 158.06259970688686, Total Accuracy: 66.57%
Accuracy per class - G1: 0.00%, G2: 78.92%, G3: 69.70%
Epoch 7, Loss: 0.05995559733774927
Validation Loss: 203.93657097796148, Total Accuracy: 64.48%
Accuracy per 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          G2_Acc ██▃▁▃▃▅▄▄▅▂▂▃▃▄▄▂▂▄▄▃▁▄▃▃▂▂▂▂▃▁▁▂▁▃▂▂▂▂▂
wandb:          G3_Acc ▁▁▇█▇██▇▇█████████▆▇████████████████████
wandb:  Total Accuracy ▁▁▇▇▇▇█▇▇█▇▇▇▇██▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▇▇▇▇▇▇
wandb: Validation Loss ▁▁▂▂▃▃▄▅▆▆▇▆▆▆▆▆▆▇▆▆▆▆▇▇▇▇█▇▇▇▇██▇██████
wandb: 
wandb: Run summary:
wandb:          G1_Acc 0.0
wandb:          G2_Acc 81.3253
wandb:          G3_Acc 65.15152
wandb:  Total Accuracy 65.97015
wandb: Validation Loss 918.4348
wandb: 
wandb: 🚀 View run microsoft/resnet-50 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/5na9vyqd
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240916_150705-5na9vyqd

### Resnet-18

In [14]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-18')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [16]:
config={
    "learning_rate": 0.0001,
    "architecture": "microsoft/resnet-18",
    "epochs": 70,
    "weight_decay": 0.01,
    "reduce_lr_factor": 0.25,
    "patience": 20
    }
train(model, config, run_name = config["architecture"])

wandb: wandb version 0.18.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240916_153454-4fr6000c
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-18
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/4fr6000c


Epoch 1, Loss: 0.097293415656572
Validation Loss: 1.4849458501081576, Total Accuracy: 54.33%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 43.18%
Epoch 2, Loss: 0.0037178687586371475
Validation Loss: 1.6059184797023507, Total Accuracy: 58.81%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 54.55%
Epoch 3, Loss: 0.002387137589620478
Validation Loss: 1.5437354690352962, Total Accuracy: 64.18%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 68.18%
Epoch 4, Loss: 0.056638968474645585
Validation Loss: 3.471437842161818, Total Accuracy: 59.10%
Accuracy per class - G1: 0.00%, G2: 76.51%, G3: 53.79%
Epoch 5, Loss: 0.030163416904947046
Validation Loss: 1.6726095738757232, Total Accuracy: 62.69%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 64.39%
Epoch 6, Loss: 0.0014556941176932822
Validation Loss: 1.8401587274539368, Total Accuracy: 62.99%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 65.15%
Epoch 7, Loss: 0.0005347376043371494
Validation Loss: 2.0107600367839704, Total Accuracy: 62.99%

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          G2_Acc █████████▁██████████████████████████████
wandb:          G3_Acc ▄▅▅▆▆▆▅█▅▁█▇▆▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▆▆▆▆▆▆▆▆▆
wandb:  Total Accuracy ▆▇▇▇▇▇▇█▇▁█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
wandb: Validation Loss ▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:          G1_Acc 0.0
wandb:          G2_Acc 75.3012
wandb:          G3_Acc 65.15152
wandb:  Total Accuracy 62.98507
wandb: Validation Loss 2.27749
wandb: 
wandb: 🚀 View run microsoft/resnet-18 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/4fr6000c
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240916_153454-4fr6000c/

### Resnet-34

In [17]:

model = ResNetForImageClassification.from_pretrained('microsoft/resnet-34')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [19]:
config={
    "learning_rate": 0.0001,
    "architecture": "microsoft/resnet-34",
    "epochs": 70,
    "weight_decay": 0.01,
    "reduce_lr_factor": 0.5,
    "patience": 20
    }
train(model, config, run_name = config["architecture"])


wandb: wandb version 0.18.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240916_155133-k8x2rfne
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-34
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/k8x2rfne


Epoch 1, Loss: 0.12671266874531284
Validation Loss: 1.6998469250446016, Total Accuracy: 65.97%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 72.73%
Epoch 2, Loss: 0.00602293118976781
Validation Loss: 1.8177035400982609, Total Accuracy: 65.67%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 71.97%
Epoch 3, Loss: 0.038362132881755114
Validation Loss: 1.6518114805221558, Total Accuracy: 65.07%
Accuracy per class - G1: 0.00%, G2: 54.22%, G3: 96.97%
Epoch 4, Loss: 0.03396857356387449
Validation Loss: 1.3933376465204426, Total Accuracy: 66.27%
Accuracy per class - G1: 0.00%, G2: 90.96%, G3: 53.79%
Epoch 5, Loss: 0.009481658723315276
Validation Loss: 1.492658874443309, Total Accuracy: 68.66%
Accuracy per class - G1: 0.00%, G2: 77.11%, G3: 77.27%
Epoch 6, Loss: 0.0009696401832318063
Validation Loss: 1.7399519169967705, Total Accuracy: 67.76%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 77.27%
Epoch 7, Loss: 0.00029882556039713865
Validation Loss: 1.6425233399707146, Total Accuracy: 75.82%

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          G2_Acc ▇▇█▇▇▇▇█▇▇▇▇▇▇▇▇▇▇███▇█▇▇▇▇▇▇▇▇▁▇▇▇▇▇▇██
wandb:          G3_Acc ▄▄▂▅▆▆▆▁▃▃▃▃▃▅▅▅▆▅▆▅▃▅▅▅▅▅▅▅▃▄▄█▃▃▂▃▃▂▄▃
wandb:  Total Accuracy ▆▆▆▆▇▇▇▅▅▆▆▆▆▆▆▇▇▆█▇▆▇▇▇▇▇▆▆▆▆▆▁▅▅▅▅▅▅▇▇
wandb: Validation Loss ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:          G1_Acc 0.0
wandb:          G2_Acc 89.75904
wandb:          G3_Acc 64.39394
wandb:  Total Accuracy 69.85075
wandb: Validation Loss 1.10523
wandb: 
wandb: 🚀 View run microsoft/resnet-34 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/k8x2rfne
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240916_155133-k8x2rfne